In [1]:
import pandas as pd
import numpy as np
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_df

In [4]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(education_folder_path),'מוסדות בירושלים 2020')
JLM=remove_spaces_in_columns(JLM)

In [5]:
#קאורדינטות של מוסדות חינוך בכל הארץ ממשרד החינוך
moe_mosdot_coordinates_2022=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(education_folder_path),'moe_mosdot_coordinates_2022')
moe_mosdot_coordinates_2022=remove_spaces_in_columns(moe_mosdot_coordinates_2022)

In [6]:
JLM.rename(columns={'סמל_חינוך': 'סמל_מוסד'}, inplace=True)

JLM = JLM.drop(columns=['אגף', 'פיקוח', 'מעמד_משפטי', "מס'_תלמידים_ז-ט", "מס'_תלמידים_י-יד", 'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'Unnamed:_18', 'Unnamed:_19'])

JLM=JLM[JLM['שלב_חינוך'] != 'גני ילדים']

In [7]:
moe_mosdot_coordinates_2022.rename(columns={'SEMEL_MOSAD': 'סמל_מוסד'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'SHEM_MOSAD': 'שם מוסד'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'ITM_X': 'coordinate_x'}, inplace=True)
moe_mosdot_coordinates_2022.rename(columns={'ITM_Y': 'coordinate_y'}, inplace=True)

moe_mosdot_coordinates_2022 = moe_mosdot_coordinates_2022.drop(columns=['UTM_X', 'UTM_Y', 'RAMAT_DIYUK_MIKUM'])

In [8]:
rows_with_zero = JLM[JLM['coordinate_x'] == 0]
JLM = JLM[JLM['coordinate_x'] != 0]

# מיזוג הטבלאות לפי העמודה "סמל_מוסד"
rows_with_zero = pd.merge(rows_with_zero, moe_mosdot_coordinates_2022[['סמל_מוסד', 'coordinate_x', 'coordinate_y']],
                     on='סמל_מוסד', how='left')

# עדכון העמודות המקוריות
rows_with_zero['coordinate_x'] = rows_with_zero['coordinate_x_y']
rows_with_zero['coordinate_y'] = rows_with_zero['coordinate_y_y']

# מחיקת העמודות המיותרות
rows_with_zero = rows_with_zero.drop(['coordinate_x_x', 'coordinate_y_x', 'coordinate_x_y', 'coordinate_y_y'], axis=1)

In [9]:
# מיזוג הטבלאות
JLM=pd.concat([JLM, rows_with_zero], ignore_index=True)

In [10]:
JLM['סמל_מוסד'] = JLM['סמל_עירייה']
JLM.drop(columns='סמל_עירייה', inplace=True)

In [11]:
is_unique = JLM['סמל_מוסד'].is_unique
is_unique

True

In [12]:
# הגדרת מיפוי של כיתות לטווחים - שימוש באותיות עבריות לטווח הכיתות
class_map = {'א': 1, 'ב': 2, 'ג': 3, 'ד': 4, 'ה': 5, 'ו': 6, 'ז': 7, 'ח': 8, 'ט': 9, 'י': 10, 'יא': 11, 'יב': 12, 'יג': 13, 'יד': 14}
reverse_class_map = {v: k for k, v in class_map.items()}

# המרת ערכי כיתות מספריים
JLM['מכיתה_num'] = JLM['מכיתה'].map(class_map)
JLM['עד_כיתה_num'] = JLM['עד_כיתה'].map(class_map)

# לולאה על כל שורה בטבלה
for index, row in JLM.iterrows():
    total_students = row['סה"כ_תלמידים']
    num_classes = row['מספר_כיתות']
    
    # טווח כיתות
    class_range = list(range(row['מכיתה_num'], row['עד_כיתה_num'] + 1))
    num_classes_in_range = len(class_range)
    
    # חלוקת סיבוב ראשון - 15 תלמידים לכל כיתה
    first_round_allocation = {f'כיתה_{reverse_class_map[i]}': min(15, total_students) for i in class_range}
    total_students -= sum(first_round_allocation.values())
    
    # חלוקה שווה של התלמידים שנותרו
    remaining_students_per_class = total_students // num_classes_in_range
    remainder = total_students % num_classes_in_range
    
    # הוספת החלוקה השווה לתוך התוצאות של כל כיתה
    for i in class_range:
        class_name = f'כיתה_{reverse_class_map[i]}'
        first_round_allocation[class_name] += remaining_students_per_class
        if remainder > 0:
            first_round_allocation[class_name] += 1
            remainder -= 1
    
    # הוספת העמודות המתאימות ל-DataFrame
    for i in class_range:
        JLM.loc[index, f'כיתה_{reverse_class_map[i]}'] = first_round_allocation[f'כיתה_{reverse_class_map[i]}']

# כעת בודקים אם ממוצע התלמידים בכל כיתה הוא לפחות 15

# יוצרים רשימה של העמודות שנוצרו עבור הכיתות (שמתחילות ב-"כיתה_")
class_columns = [col for col in JLM.columns if col.startswith('כיתה_') and col.startswith('כיתה_')]

# בודקים את הממוצע של כל כיתה ומוחקים שורות שלא עומדות בתנאי
JLM['ממוצע_תלמידים_לכיתה'] = JLM[class_columns].mean(axis=1)
# JLM = JLM[JLM['ממוצע_תלמידים_לכיתה'] >= 15]

# הסרת עמודת העזר ממוצע_תלמידים_לכיתה
# JLM = JLM.drop(columns=['ממוצע_תלמידים_לכיתה'])

JLM = JLM.drop(['מכיתה_num', 'עד_כיתה_num'], axis=1) 

JLM.head()

,סמל_מוסד,שם_מוסד,שלב_חינוך,סוג_חינוך,מכיתה,עד_כיתה,מספר_כיתות,"סה""כ_תלמידים",coordinate_x,coordinate_y,כיתה_א,כיתה_ב,כיתה_ג,כיתה_ד,כיתה_ה,כיתה_ו,כיתה_ז,כיתה_ח,כיתה_ט,כיתה_י,כיתה_יא,כיתה_יב,כיתה_יג,כיתה_יד,ממוצע_תלמידים_לכיתה
0,21321.00,"מ""מ אגרון",יסודי,רגיל,א,ו,8.00,230.00,216876.00,629987.00,39.00,39.00,38.00,38.00,38.00,38.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.33
1,21323.00,"מ""מ ארגנטינה הניסויי",יסודי,רגיל,א,ח,12.00,347.00,216154.00,630603.00,44.00,44.00,44.00,43.00,43.00,43.00,43.00,43.00,NaN,NaN,NaN,NaN,NaN,NaN,43.38
2,21326.00,"מ""מ בית הכרם -הישראלי",יסודי,רגיל,א,ו,17.00,534.00,218166.00,632109.00,89.00,89.00,89.00,89.00,89.00,89.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.00
3,21327.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",יסודי,רגיל,א,ו,13.00,331.00,220805.00,629266.00,56.00,55.00,55.00,55.00,55.00,55.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.17
4,21328.00,"מ""מ זלמן ארן",יסודי,רגיל,א,ו,13.00,367.00,220633.00,628408.00,62.00,61.00,61.00,61.00,61.00,61.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.17


In [13]:
# חלוקה לשכבות גיל
ages = {
    'ele': ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו'],
    'mid': ['כיתה_ז', 'כיתה_ח', 'כיתה_ט'],
    'high': ['כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']
}

# חישוב מספר התלמידים לכל שכבה
JLM['ele_stu'] = JLM[ages['ele']].sum(axis=1)
JLM['mid_stu'] = JLM[ages['mid']].sum(axis=1)
JLM['high_stu'] = JLM[ages['high']].sum(axis=1)

JLM_1 = JLM.drop(['שלב_חינוך', 'כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד', 'ממוצע_תלמידים_לכיתה', 'מספר_כיתות', 'סה"כ_תלמידים', 'מכיתה', 'עד_כיתה', 'סוג_חינוך', 'שם_מוסד'], axis=1)
JLM.to_excel('JLM.xlsx', index=False)
JLM_1.to_excel('JLM_final.xlsx', index=False)

# הצגת התוצאה
# JLM.head(400)